<a href="https://colab.research.google.com/github/aarongonzalezoxed/OTCC-MOCK-V2/blob/main/LTE_MOCK_ADULTOS_LAST_UPDATE_22_01_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#DATA MANAGEMENT / OTCC MOCK ADULTOS

import io, re, os
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

AUTO_DOWNLOAD = True
try:
    from google.colab import files
except Exception:
    files = None

print("Sube los reportes de AssessmentQ (puedes seleccionar varios; .csv, .xlsx o .xls).")
if files:
    uploaded = files.upload()
else:
    raise RuntimeError("Ejecuta esto en Google Colab.")

# ====== Tabla de máximos (MOCK) ======
max_scores_map = {
    ('A1','USE OF ENGLISH'): 400, ('A1','READING'): 200, ('A1','LISTENING'): 200, ('A1','WRITING'): 200,
    ('A2','USE OF ENGLISH'): 400, ('A2','READING'): 200, ('A2','LISTENING'): 200, ('A2','WRITING'): 200,
    ('B1','USE OF ENGLISH'): 400, ('B1','READING'): 280, ('B1','LISTENING'): 240, ('B1','WRITING'): 200,
    ('B2','USE OF ENGLISH'): 400, ('B2','READING'): 280, ('B2','LISTENING'): 240, ('B2','WRITING'): 200,
    ('C1','USE OF ENGLISH'): 400, ('C1','READING'): 200, ('C1','LISTENING'): 240, ('C1','WRITING'): 300,
    ('C2','USE OF ENGLISH'): 400, ('C2','READING'): 200, ('C2','LISTENING'): 320, ('C2','WRITING'): 300,
}

# ====== Utilidades ======
def read_any_table(fname: str, data: bytes) -> pd.DataFrame:
    lower = fname.lower()
    if lower.endswith((".xlsx", ".xls")):
        return pd.read_excel(io.BytesIO(data), header=None, dtype=str, engine="openpyxl")
    try:
        return pd.read_csv(io.BytesIO(data), header=None, sep=None, engine="python", encoding="utf-8", dtype=str)
    except Exception:
        return pd.read_csv(io.BytesIO(data), header=None, sep=None, engine="python", encoding="latin-1", dtype=str)

def extract_exam_label_B1(df_raw: pd.DataFrame) -> str:
    try:
        return str(df_raw.iat[0,1]).strip()
    except Exception:
        return ""

def parse_exam_fields(exam_label: str):
    level, group_id = None, None
    m = re.search(r'OTCC\s+MOCK\s+([ABC]\d)\b', exam_label, re.IGNORECASE)
    if m: level = m.group(1).upper()
    g = re.search(r'\(([^)]+)\)', exam_label)
    if g: group_id = g.group(1).strip()
    return level, group_id

def set_header_row4(df_raw: pd.DataFrame):
    if df_raw.shape[0] < 4:
        raise ValueError("El archivo no trae al menos 4 filas (esperada cabecera en fila 4).")
    header = df_raw.iloc[3].astype(str).tolist()
    df = df_raw.iloc[4:].copy()
    df.columns = header
    return df

# Partes / columnas (MOCK)
PARTS = {"USE OF ENGLISH": 2, "READING": 3, "LISTENING": 4, "WRITING": 5}
COL_SCORE   = "Part {n}- Score"
COL_PERCENT = "Part {n}- Score(%)"

ID_COLS = {
    "user_id": ["User Id","User ID","UserId","Userid"],
    "first":   ["First Name","Firts Name","Firstname","First name"],
    "last":    ["Last Name","Lastname","Last name"],
}
def find_first_present(df_cols, candidates):
    for c in candidates:
        if c in df_cols: return c
    lower = {c.lower(): c for c in df_cols}
    for c in candidates:
        if c.lower() in lower: return lower[c.lower()]
    return None

def to_num(x):
    try: return float(str(x).replace('%','').replace(',','').strip())
    except: return np.nan

def pct_two_dec_str(x):
    if x is None or (isinstance(x,float) and np.isnan(x)):
        return "N/A"
    return f"{round(float(x),2):.2f}%"

def sanitize_png_name(name):
    return re.sub(r'[\\/*?:\"<>|]+','_', name).strip()

def unified_csv_name():
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    return f"OTCC MOCK - UNIFICADO_{ts}.csv"

# ====== NUEVO ORDEN / NUEVAS COLUMNAS ======
CSV_COLS = [
    "User ID",
    "Test name",
    "Full name",
    "Applied level",
    "Use of English",
    "Reading",
    "Listening",
    "Writing",
    "Score Obtained",
    "Level Achieved",
    "Passing Criteria",
]

invalid_files = []
fallback_missing_max = []
master_rows, console_overall_rows = [], []
acc = {"USE OF ENGLISH": [], "READING": [], "LISTENING": [], "WRITING": [], "OVERALL": []}

# Ahora la distribución debe ser por Level Achieved (incluyendo N/A)
level_counts = {}
level_order = ["A1","A2","B1","B2","C1","C2","N/A"]

def level_one_below(level):
    base_order = ["A1","A2","B1","B2","C1","C2"]
    if level not in base_order:
        return level
    idx = base_order.index(level)
    return base_order[max(0, idx-1)]

def compute_level_achieved(applied_level, score_obtained):
    if applied_level is None or score_obtained is None or (isinstance(score_obtained,float) and np.isnan(score_obtained)):
        return None

    s = float(score_obtained)

    # C2 reglas especiales
    if applied_level == "C2":
        if s >= 80:
            return "C2"
        elif 70 <= s <= 79.999999:
            return "C1"
        else:
            return "B2"

    # A1–C1
    if s >= 70:
        return applied_level
    else:
        # si es A1, se queda A1
        if applied_level == "A1":
            return "A1"
        return level_one_below(applied_level)

def compute_passing_criteria(applied_level, score_obtained):
    if applied_level is None or score_obtained is None or (isinstance(score_obtained,float) and np.isnan(score_obtained)):
        return "N/A"

    s = float(score_obtained)
    threshold = 80.0 if (applied_level == "C2") else 70.0

    if s >= threshold:
        return "PASSED"
    else:
        # Regla especial A1: no hay nivel abajo, queda FAILED
        if applied_level == "A1":
            return "FAILED"
        return "1LB"

# ====== Proceso de todos los archivos ======
for fname, content in uploaded.items():
    try:
        df_raw = read_any_table(fname, content)
        exam_label = extract_exam_label_B1(df_raw)
        if not exam_label or not exam_label.upper().startswith("OTCC MOCK"):
            invalid_files.append(fname); continue

        level_from_name, group_id = parse_exam_fields(exam_label)
        df = set_header_row4(df_raw)

        user_id_col = find_first_present(df.columns, ID_COLS["user_id"])
        fn_col      = find_first_present(df.columns, ID_COLS["first"])
        ln_col      = find_first_present(df.columns, ID_COLS["last"])
        if not user_id_col or not fn_col or not ln_col:
            raise ValueError("Faltan columnas de identidad (User/First/Last).")

        # Overall del reporte (puede variar el encabezado en algunos archivos)
        overall_report_col = "Score (%)"
        if overall_report_col not in df.columns:
            for alt in ["Score(%)","Overall Score (%)","Total Score (%)","Overall (%)","Total (%)"]:
                if alt in df.columns:
                    overall_report_col = alt
                    break
            else:
                overall_report_col = None

        cnt = 0
        for _, row in df.iterrows():
            user_id    = str(row.get(user_id_col, "")).strip()
            first_name = str(row.get(fn_col, "")).strip()
            last_name  = str(row.get(ln_col, "")).strip()
            full_name  = (first_name + " " + last_name).strip()

            # Recalcular por habilidad
            skill_calc, skill_native = {}, {}
            for skill, part_n in PARTS.items():
                score_col = COL_SCORE.format(n=part_n)
                pct_col   = COL_PERCENT.format(n=part_n)
                raw_points     = to_num(row.get(score_col, np.nan))
                raw_pct_native = to_num(row.get(pct_col,   np.nan))
                skill_native[skill] = None if np.isnan(raw_pct_native) else raw_pct_native

                calc_pct = np.nan
                key = (level_from_name, skill)
                if level_from_name and key in max_scores_map and not np.isnan(raw_points):
                    max_sc = max_scores_map[key]
                    if max_sc and max_sc > 0:
                        calc_pct = (raw_points / max_sc) * 100.0
                if np.isnan(calc_pct):
                    calc_pct = raw_pct_native
                    if level_from_name and key not in max_scores_map:
                        fallback_missing_max.append((fname, level_from_name, skill))
                skill_calc[skill] = None if np.isnan(calc_pct) else float(round(calc_pct,4))

            vals = [v for v in skill_calc.values() if v is not None]
            overall_calc = round(float(np.mean(vals)),4) if vals else None  # <- Score Obtained

            # Overall del reporte (Score %)
            overall_report = None
            if overall_report_col is not None:
                overall_report = to_num(row.get(overall_report_col, np.nan))
                if isinstance(overall_report,float) and np.isnan(overall_report):
                    overall_report = None

            # ===== NUEVAS REGLAS =====
            applied_level = level_from_name
            level_achieved = compute_level_achieved(applied_level, overall_calc)
            passing_criteria = compute_passing_criteria(applied_level, overall_calc)

            # Fila CSV (nuevo orden / nombres)
            master_rows.append({
                "User ID": user_id,
                "Test name": exam_label,
                "Full name": full_name,
                "Applied level": applied_level,
                "Use of English": pct_two_dec_str(skill_calc.get("USE OF ENGLISH")),
                "Reading":       pct_two_dec_str(skill_calc.get("READING")),
                "Listening":     pct_two_dec_str(skill_calc.get("LISTENING")),
                "Writing":       pct_two_dec_str(skill_calc.get("WRITING")),
                "Score Obtained": pct_two_dec_str(overall_calc),
                "Level Achieved": level_achieved if level_achieved else "N/A",
                "Passing Criteria": passing_criteria,
            })
            cnt += 1

            # Consola: mostramos ambos overalls
            ours_str   = pct_two_dec_str(overall_calc)
            report_str = pct_two_dec_str(overall_report)
            if overall_calc is None or overall_report is None:
                status = "N/A"
            else:
                status = "COINCIDE" if round(overall_calc,2) == round(overall_report,2) else "NO"

            console_overall_rows.append({
                "Test name": exam_label,
                "User ID": user_id,
                "Full name": full_name,
                "Score Obtained (CSV%)": ours_str,
                "OVERALL REPORTE (Score %)": report_str,
                "STATUS": status
            })

            # Promedios (primer gráfico usa Score Obtained)
            for sk in ["USE OF ENGLISH","READING","LISTENING","WRITING"]:
                if skill_calc.get(sk) is not None: acc[sk].append(skill_calc[sk])
            if overall_calc is not None: acc["OVERALL"].append(overall_calc)

            # Distribución (segundo gráfico usa Level Achieved) -> incluir N/A
            lvl_key = level_achieved if level_achieved else "N/A"
            level_counts[lvl_key] = level_counts.get(lvl_key,0)+1

        print(f"OK: {fname} → {cnt} filas integradas. [{exam_label}]")

    except Exception as e:
        print(f"ERROR en {fname}: {e}")
        invalid_files.append(fname)

# ====== Exportar CSV único ======
if master_rows:
    df_master = pd.DataFrame.from_records(master_rows, columns=CSV_COLS)
    out_csv = unified_csv_name()
    df_master.to_csv(out_csv, index=False, encoding='utf-8-sig')
    print(f"\nCSV UNIFICADO generado: {out_csv} — {len(df_master)} filas.")
    if files and AUTO_DOWNLOAD:
        try: files.download(out_csv)
        except Exception as e: print(f"No se pudo descargar {out_csv}: {e}")
else:
    print("\nNo se generó CSV unificado (sin filas válidas).")

# ====== Consola: Score Obtained (CSV%) vs Score (%) ======
if console_overall_rows:
    df_console = pd.DataFrame(
        console_overall_rows,
        columns=["Test name","User ID","Full name","Score Obtained (CSV%)","OVERALL REPORTE (Score %)","STATUS"]
    )
    print("\n=== Verificación — Score Obtained (CSV%) vs OVERALL REPORTE (Score %) ===")
    print(df_console.to_string(index=False))

# ====== Gráficos (barras + donut) ======
def save_and_download(fig, fname_png, dpi=300):
    fig.tight_layout(); fig.savefig(fname_png, dpi=dpi, bbox_inches="tight"); plt.close(fig)
    if files and AUTO_DOWNLOAD:
        try: files.download(fname_png)
        except Exception as e: print(f"No se pudo descargar {fname_png}: {e}")

bar_colors = ["#0d47a1", "#1565c0", "#1976d2", "#1e88e5", "#64b5f6"]
pie_colors = ["#0d47a1", "#1565c0", "#1976d2", "#1e88e5", "#42a5f5", "#90caf9", "#b0bec5"]

uoe_avg  = np.mean(acc["USE OF ENGLISH"]) if acc["USE OF ENGLISH"] else np.nan
read_avg = np.mean(acc["READING"])        if acc["READING"] else np.nan
lis_avg  = np.mean(acc["LISTENING"])      if acc["LISTENING"] else np.nan
wri_avg  = np.mean(acc["WRITING"])        if acc["WRITING"] else np.nan
ovr_avg  = np.mean(acc["OVERALL"])        if acc["OVERALL"] else np.nan  # <- Score Obtained promedio

cats = ["Use of English","Reading","Listening","Writing","Overall"]
vals = [uoe_avg, read_avg, lis_avg, wri_avg, ovr_avg]

fig1 = plt.figure(figsize=(11,5.5))
ax = fig1.gca()
bars = ax.bar(cats, vals, color=bar_colors)
ax.set_ylim(0,100)
ax.set_ylabel("Percentage (%)")
ax.set_title("OTCC MOCK — Integrated — Average per skill and overall. (MOCK)", fontsize=14, pad=12)
for b,v in zip(bars, vals):
    if v==v:
        ax.text(b.get_x()+b.get_width()/2, b.get_height()+1, f"{round(v,2):.2f}%", ha='center', va='bottom', fontsize=10)
save_and_download(fig1, sanitize_png_name("OTCC MOCK — Integrated - Average per skill and overall (MOCK).png"))

# Donut: Distribution by English language level -> Level Achieved + N/A
pairs_unsorted = [(lvl, cnt) for lvl, cnt in level_counts.items() if cnt>0]
if not pairs_unsorted and master_rows:
    nv = pd.Series([r["Level Achieved"] for r in master_rows]).astype(str)
    # convertir vacíos o nan a N/A
    nv = nv.replace({"None":"N/A","nan":"N/A","NaN":"N/A","": "N/A"})
    for lvl, cnt in nv.value_counts().items(): pairs_unsorted.append((lvl, int(cnt)))

if pairs_unsorted:
    def order_key(lvl):
        return level_order.index(lvl) if lvl in level_order else 999

    pairs = sorted(pairs_unsorted, key=lambda x: order_key(x[0]))
    labels, sizes = zip(*pairs); total = int(sum(sizes))

    fig2 = plt.figure(figsize=(10,7)); ax2 = fig2.gca()
    colors = pie_colors[:len(sizes)] if len(pie_colors) >= len(sizes) else None
    wedges, _ = ax2.pie(
        sizes, startangle=0, counterclock=False,
        colors=colors,
        wedgeprops=dict(width=0.50, edgecolor='white', linewidth=1.2)
    )
    ax2.text(0, 0, f"Total\n{total}", ha='center', va='center', fontsize=13)
    ax2.axis('equal')
    ax2.set_title("OTCC MOCK — Integrated — Distribution by English language level (MOCK)", fontsize=16, pad=14)

    inner_r = 1.0 - 0.50
    r_text  = inner_r + 0.25
    for w, lbl, sz in zip(wedges, labels, sizes):
        theta = (w.theta2 + w.theta1) / 2.0
        ang   = np.deg2rad(theta)
        x = r_text * np.cos(ang); y = r_text * np.sin(ang)
        pct = 100.0 * sz / total; fz = 11 if pct >= 6 else 9
        ax2.text(x, y, f"{lbl} {pct:.1f}%", ha='center', va='center',
                 fontsize=fz, fontweight='bold', color='black')

    legend_text = "\n".join([f"{lvl} = {cnt} persons at this level" for lvl, cnt in pairs])
    fig2.text(0.98, 0.02, legend_text, ha='right', va='bottom',
              bbox=dict(boxstyle="round,pad=0.6", facecolor="white", alpha=0.95, linewidth=0.8),
              fontsize=13)

    save_and_download(fig2, sanitize_png_name("OTCC MOCK — Integrated - Distribution by English language level (MOCK).png"))

# ====== Calidad ======
if invalid_files:
    print("\nArchivos descartados por no ser 'OTCC MOCK' (o con errores):")
    for x in invalid_files: print("  -", x)
if fallback_missing_max:
    print("\nAviso: se usó % nativo (Part X- Score(%)) por falta de max_score para:")
    seen=set()
    for fname, level, skill in fallback_missing_max:
        key=(level,skill)
        if key not in seen:
            print(f"  - level={level}, skill={skill}")
            seen.add(key)

print("\nListo.")


Sube los reportes de AssessmentQ (puedes seleccionar varios; .csv, .xlsx o .xls).


Saving 20260114_AssignmentReport_OTCC MOCK B1 (S0_1009).xlsx to 20260114_AssignmentReport_OTCC MOCK B1 (S0_1009).xlsx
Saving 20260114_AssignmentReport_OTCC MOCK B2 (S0_1009).xlsx to 20260114_AssignmentReport_OTCC MOCK B2 (S0_1009).xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


OK: 20260114_AssignmentReport_OTCC MOCK B1 (S0_1009).xlsx → 21 filas integradas. [OTCC MOCK B1 (S0_1009)]


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Listo.
